## Trading Strategy Explorer
Trading strategy exploration and backtesting project. 
Change the strategy and discover the performance on your dataset.

###  Load data 

In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
file_path = sys.path[0] # give me the working directory
file_path

'/home/kobayashi/Documents/in progress 💚/RedPanda 🐼/RedPandaGenerator/notebooks'

In [3]:
df = pd.read_csv("ETHUSD_2021_minute.csv",skiprows=1) # load my csv dataset 
                     #index_col=0, 
                     #parse_dates=True,
                     #infer_datetime_format=True)

FileNotFoundError: [Errno 2] No such file or directory: 'ETHUSD_2021_minute.csv'


### Exploratory analysis 

In [ ]:
df.head() # print the first 5 rows

In [ ]:
cols = [i for i in list(df.columns)] # show me all the columns
print(cols)

In [ ]:
df.isna().sum()  #check for nan values


### Data Preprocessing

In [ ]:
df = df.drop(['symbol'],axis=1) # drop this column
df = df.drop(['Volume USD'],axis=1)

In [ ]:
#df['date']=df['date'].apply(lambda x: str(x).replace('00:00:00',''))  # drop this column 

In [ ]:
# set date as dataframe index, empty dates can
# df=df.set_index('date') # set date as index

In [ ]:
# My first value is the last one in time order, so I have to reorder my dataset.
df2 = df.sort_values(['date']) # SORT BY DATE

In [ ]:
df2.head(1) # first row

In [ ]:
df2.tail(1) # last row

## Backtesting

In [ ]:
import datetime

In [ ]:
def plot_signals(df,plot_open_signal,plot_close_signal):
    # PLOT SIGNALS and EQUITY
    f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharey=True)
    f.set_figheight(10)
    f.set_figwidth(12)
    
    # OPEN SIGNAL
    ax1.vlines(plot_open_signal,ymin=0,ymax=1,colors='green')
    ax1.set_title('open signal')

    ax1.set_ylim(0, 1)
    ax1.set_xlim(0,max(plot_open_signal)+5)
    x_axis1 = ax1.axes.get_xaxis()
    x_axis1.set_visible(False)
    y_axis1 = ax1.axes.get_yaxis()
    y_axis1.set_visible(False)
    
    # CLOSE SIGNAL
    ax2.vlines(plot_close_signal,ymin=0,ymax=1,colors='red')
    ax2.set_title('close signal')
    x_axis2 = ax2.axes.get_xaxis()    
    x_axis2.set_visible(False)
    y_axis2 = ax2.axes.get_yaxis()
    y_axis2.set_visible(False)
    ax2.set_ylim(0, 1)
    ax2.set_xlim(0,max(plot_open_signal)+5)
    
    
    # PRICES
    ax3.plot(df.index, df.close)
    ax3.set_title('ethereum / USDollar')
    ax3.set_xticks(df.index[::5])
    ax3.set_xticklabels(df.index[::5], rotation=45)
    ax3.set_ylim(min(df.close)*0.97, max(df.close)*1.03)

    plt.show()

In [ ]:
def plot_equity(equity_history):
    f, ax1 = plt.subplots(1, 1, sharey=True)
    f.set_figheight(5)
    f.set_figwidth(12)
    plt.plot(equity_history)
    plt.title('Equity')
    plt.show()

In [ ]:
######################################
# MY DATA ############################
take_profit = 2      # in percentage
stop_loss = 1        # in percentage
my_investment = 10_000  # capital
laverage = 1            # laverage
######################################


plot_open_signal=[] # open signal
plot_close_signal=[] # close signal
equity_history=[]



def backtest(df, capital):
    from copy import deepcopy
    equity = deepcopy(capital)
    open_operation = False
    buffer_period = 200
    
    for idx in range(len(df)):
        
        Date = df['date'].iloc[idx]
        Open = df['open'].iloc[idx]
        High = df['high'].iloc[idx]
        Low  = df['low'].iloc[idx]
        Close  = df['close'].iloc[idx]
        
        if idx > buffer_period: #and int(idx) < len(df.values)-1:  # buffer
            
            if open_operation==False:
                
                ##################################################################################
                # Open Long position #############################################################
                
                if Close < df['close'].iloc[idx-240] and Close < df['close'].iloc[idx-3360]: 
                    
                ##################################################################################
                
                    open_operation = True
                    open_price = Close
                    open_date = idx
                    size_order = equity/open_price    

                    print('\nopen long position  ',Date,'| price opening operation:',Close,'\t',
                                                    '| equity',round(equity,2),'| size order',size_order)

                    plot_open_signal.append(idx)

            actual_price = Close
            timedelta = (idx-open_date)/60 # minutes between open operation time and now 
                
            if open_operation==True: # Open buy price
                
                gain =  size_order * (open_price - actual_price)         
                real_TP = take_profit/100*equity
                real_SL = stop_loss/100*equity
                # print(gain)
                # print(gain,real_TP,real_SL)
                
                
                ##################################################################################
                # Close Long position ############################################################
                
                if ( (gain >= real_TP) or (gain <= -real_SL) ) and timedelta > 240: 
                    
                ##################################################################################
                
                    equity += gain
                    open_operation = False
                    
                    plot_close_signal.append(idx)
                    
                    print('close long position ',Date,'| price closing operation:',Close,'\t',
                          '| equity',round(equity,2),'| gain ',round(gain,3),'$')
                    
            if equity <= equity*0.5:
                break
                
            equity_history.append(equity)
    
   
    return equity_history, plot_open_signal, plot_close_signal                  


equity,plot_open_signal, plot_close_signal = backtest(df2[:3000],capital=my_investment)

## Plot data

In [ ]:
plot_equity(equity)

#### ATTENTION! DONT TRY THIS IF YOUR DATASET HAVE MORE THAN 10 000 datapoints, huge processing time

In [ ]:
#plot_signals(df=df2, plot_open_signal=plot_open_signal ,plot_close_signal=plot_close_signal )